In [1]:
import pickle
import pandas

with open('data_pickle_format/synethetic_vacancies_final.pickle', 'rb') as f:
    df = pickle.load(f)    

counts_1_4 = df.loc[df['women_proportion'] <= 0.25]
counts_2_4 = df.loc[(df['women_proportion'] > 0.25) & (df['women_proportion'] <= 0.5)]
counts_3_4 = df.loc[(df['women_proportion'] > 0.5) & (df['women_proportion'] <= 0.75)]
counts_4_4 = df.loc[df['women_proportion'] > 0.75]

print(counts_1_4.count())
print(counts_2_4.count())
print(counts_3_4.count())
print(counts_4_4.count())

job_description     28
women_proportion    28
dtype: int64
job_description     152
women_proportion    152
dtype: int64
job_description     155
women_proportion    155
dtype: int64
job_description     31
women_proportion    31
dtype: int64


In [2]:
df['title'] = df['job_description'].str.extract(
    r'(?:Job\s*Title|Title):\s*(.*?)(?:\.|\n)\s*(?:Job\s*Description|Description):', 
    expand=False
)
df['description'] = df['job_description'].str.extract(r'(?:Job\s*Description|Description):\s*(.*)', expand=False)



In [14]:
import spacy

nlp = spacy.load("en_core_web_sm")

text = df['description'][0]
doc = nlp(text)

filter_and_lemma = [token.lemma_ for token in doc if not (token.is_stop or token.is_punct)]


print(filter_and_lemma)

['look', 'helpful', 'knowledgeable', 'Support', 'Technician', 'point', 'contact', 'technical', 'assistance', 'role', 'require', 'balance', 'independent', 'problem', 'solve', 'excellent', 'interpersonal', 'skill', 'responsible', 'diagnose', 'resolve', 'hardware', 'software', 'network', 'issue', 'employee', 'efficiency', 'patience', 'autonomy', 'manage', 'support', 'ticket', 'prioritize', 'task', 'ensure', 'timely', 'resolution', 'technical', 'expertise', 'analytical', 'approach', 'key', 'success', 'time', 'highly', 'collaborative', 'role', 'interact', 'employee', 'department', 'strong', 'communication', 'empathy', 'crucial', 'provide', 'outstanding', 'customer', 'service', 'support', 'team', 'commit', 'create', 'positive', 'helpful', 'work', 'environment', 'support', 'professional', 'growth', 'training', 'development', 'opportunity', 'seek', 'candidate', 'self', 'motivate', 'problem', 'solver', 'friendly', 'approachable', 'team', 'member', 'work', 'ensure', 'entire', 'organization', 'op

In [4]:
from nltk import FreqDist

positive_freq_dic=FreqDist(counts_1_4['job_description'])
negative_freq_dic=FreqDist(counts_2_4['job_description'])

display( positive_freq_dic, negative_freq_dic)

FreqDist({"Job Title: user interface developer. Job Description: We are looking for a dedicated and creative User Interface (UI) Developer to join our front-end development team. This role is focused on building the user interfaces for our web and mobile applications, ensuring that they are both visually appealing and highly functional. A high level of dedication is required to create UIs that are not only beautiful but also intuitive and easy to use. The ideal candidate will be a passionate and detail-oriented developer with a strong understanding of UI development and a keen eye for design. You will be responsible for turning our UI/UX designs into fully functional user interfaces, writing clean and efficient code, and working closely with our designers and back-end developers. This is a full-time, in-office position that requires close collaboration with our design and development teams. We do not offer remote work for this role to ensure a highly collaborative and creative developm

FreqDist({"Title: Information Security Manager. Description: We are looking for a driven and strategic Information Security Manager to lead our security operations and governance functions. This is a leadership role for a focused professional who can take ownership of our overall security posture and drive a culture of security awareness. You will have the autonomy to manage the security team, oversee risk assessments, and direct incident response efforts. In our fast-paced environment, you are accountable for protecting the organization's information assets. We need a self-directed leader with strong analytical skills and the ability to make decisive, risk-based judgments. This is a demanding role for an individual who is motivated by building a high-performance security program. Requirements: Proven experience in an information security leadership or management role. A deep, analytical understanding of security frameworks, risk management, and incident response. The ability to work i